In [ ]:
import pandas as pd
from datetime import datetime

df = pd.DataFrame(data)

# Convert 'Time' column to datetime
df['Time'] = pd.to_datetime(df['Time'])

# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Convert 'timestamp' column to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Sort DataFrame by 'Date', 'Time', and 'timestamp'
df = df.sort_values(by=['Date', 'Time', 'timestamp'])

# Function to calculate Feature 1 - 'login'
def calculate_login(row):
    if row['EventCode'] == 4801:
        return 1
    elif row['EventCode'] == 4800:
        return 0
    else:
        return '-'

# Calculate 'login' feature using the function
df['login'] = df.apply(calculate_login, axis=1)

# Function to calculate Feature 2 - 'duration'
def calculate_duration(df):
    login_start = None
    login_end = None
    durations = []

    for index, row in df.iterrows():
        if row['login'] == 1:
            login_start = row['timestamp']
        elif row['login'] == 0:
            login_end = row['timestamp']
            if login_start is not None:
                duration = login_end - login_start
                durations.append(duration)
                login_start = None
                login_end = None
        else:
            pass

    return durations

# Calculate 'duration' feature using the function
df['duration'] = calculate_duration(df)

print(df)


In [ ]:

df = pd.DataFrame(data)

# Convert 'Time' column to datetime
df['Time'] = pd.to_datetime(df['Time'])

# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Convert 'timestamp' column to datetime using strptime
df['timestamp'] = df['timestamp'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

# Sort DataFrame by 'Date', 'Time', and 'timestamp'
df = df.sort_values(by=['Date', 'Time', 'timestamp'])

# Function to calculate Feature 1 - 'login'
def calculate_login(row):
    if row['EventCode'] == 4801:
        return 1
    elif row['EventCode'] == 4800:
        return 0
    else:
        return '-'

# Calculate 'login' feature using the function
df['login'] = df.apply(calculate_login, axis=1)

# Function to calculate Feature 2 - 'duration'
def calculate_duration(df):
    login_start = None
    login_end = None
    durations = []

    for index, row in df.iterrows():
        if row['login'] == 1:
            login_start = row['timestamp']
        elif row['login'] == 0:
            login_end = row['timestamp']
            if login_start is not None:
                duration = login_end - login_start
                durations.append(duration)
                login_start = None
                login_end = None
        else:
            pass

    return durations

# Calculate 'duration' feature using the function
df['duration'] = calculate_duration(df)

print(df)


In [ ]:


df = pd.DataFrame(data)

# Convert 'Time' column to datetime
df['Time'] = pd.to_datetime(df['Time'])

# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Convert 'timestamp' column to datetime using strptime
df['timestamp'] = df['timestamp'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

# Sort DataFrame by 'Date', 'Time', and 'timestamp'
df = df.sort_values(by=['Date', 'Time', 'timestamp'])

# Function to calculate Feature 1 - 'login'
def calculate_login(row):
    if row['EventCode'] == 4801:
        return 1
    elif row['EventCode'] == 4800:
        return 0
    else:
        return '-'

# Calculate 'login' feature using the function
df['login'] = df.apply(calculate_login, axis=1)

# Calculate 'duration' feature using lambda function
df['duration'] = df.apply(lambda x: (x['timestamp'] - df[df['timestamp'] < x['timestamp']].loc[df['login'].eq(1)].iloc[-1]['timestamp']) if x['login'] == 0 else None, axis=1)

print(df)


In [ ]:

# Visualize
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='Source_computer', hue='Account_Name')
plt.title('Login Events by Source Computer')
plt.xlabel('Source Computer')
plt.ylabel('Count')
plt.legend(title='Account Name')
plt.show()

In [ ]:
df['Login Connection'] = df.groupby(['Account_Name', 'Source_computer']).ngroup().add(1).astype(str)
df['Login Connection'] = df['Account_Name'] + '_' + df['Source_computer']

In [ ]:
# Group by 'Account_Name' and 'Date' to count logins per day
login_counts = df.groupby(['Account_Name', 'Date'])['login'].sum().reset_index()

# Plotting
plt.figure(figsize=(10, 6))
sns.barplot(data=login_counts, x='Date', y='login', hue='Account_Name')
plt.title('Number of Logins by Stan and Jon per Day')
plt.xlabel('Date')
plt.ylabel('Number of Logins')
plt.xticks(rotation=45)
plt.legend(title='Account Name')
plt.tight_layout()
plt.show()

In [ ]:
# Find first login and last logout per day for each account
first_login = df[df['login'] == 1].groupby(['Account_Name', 'Date'])['timestamp'].min().reset_index()
last_logout = df[df['login'] == 0].groupby(['Account_Name', 'Date'])['timestamp'].max().reset_index()

# Merge first login and last logout DataFrames
login_logout = pd.merge(first_login, last_logout, on=['Account_Name', 'Date'], suffixes=('_first_login', '_last_logout'))

# Calculate duration between first login and last logout per day
login_logout['Duration'] = login_logout['timestamp_last_logout'] - login_logout['timestamp_first_login']

# Convert duration to hours
login_logout['Duration_hours'] = login_logout['Duration'] / pd.Timedelta(hours=1)

print(login_logout)